In [42]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

In [43]:
CATEGORIES = ["action", "adventure", "animation", "biography", "comedy", "crime", "documentary", "drama", "family",
              "fantasy", "film_noir", "history", "horror", "music", "musical", "mystery", "romance", "sci_fi",
              "short", "sport", "superhero", "thriller", "war", "western"]

In [44]:
df = pd.read_csv("./data/preprocessed.csv")
df.head()

,title,description,action,adventure,animation,biography,comedy,crime,documentary,drama,...,mystery,romance,sci_fi,short,sport,superhero,thriller,war,western,poster_path
0,'71,"In 1971, a young and disoriented British soldi...",1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,1,0,./data/war/_71.jpg
1,'83,"On June 25, 1983, the Lord's Cricket Ground wi...",0,0,0,1,0,0,0,1,...,0,0,0,0,1,0,0,0,0,./data/sport/_83.jpg
2,'Allo 'Allo!,"In France during World War II, René Artois run...",0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,./data/history/_allo__allo_.jpg
3,10 Cloverfield Lane,A young woman is held in an underground bunker...,0,0,0,0,0,0,0,1,...,1,0,1,0,0,0,0,0,0,./data/sci_fi/10_cloverfield_lane.jpg
4,10 Things I Hate About You,"A pretty, popular teenager can't go out on a d...",0,0,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,0,./data/comedy/10_things_i_hate_about_you.jpg


In [45]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [ ]:
title_score = { genre : [] for genre in CATEGORIES}

for title in tqdm(df.title.values):
    result = classifier(title, CATEGORIES, multi_label=True)
    result_genres = result["labels"]
    result_score = result["scores"]
    for idx, label in enumerate(result_genres):
        title_score[label].append(result_score[idx])

title_score

  0%|          | 14/4188 [00:55<4:37:32,  3.99s/it]

In [ ]:
description_score = { genre : [] for genre in CATEGORIES}

for description in tqdm(df.description.values):
    result = classifier(description, CATEGORIES, multi_label=True)
    result_genres = result["labels"]
    result_score = result["scores"]
    for idx, label in enumerate(result_genres):
        description_score[label].append(result_score[idx])

description_score

In [ ]:
for label, value_list in title_score.items():
    df[f"label_{label}"] = value_list

In [ ]:
for label, value_list in description_score.items():
    df[f"description_{label}"] = value_list

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.drop(columns=["title", "description"], inplace=True)

In [ ]:
df.to_csv("./data/preprocessed_text_extracted.csv", index=False)